In [ ]:
# default_exp core

# This is MyLearning Module

> API details.

First, some imports.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import json
from datetime import datetime, date, time, timedelta

* json : JSON encoder and decoder - https://docs.python.org/3/library/json.html 

## Here is the core 

In [ ]:
#export
def say_hello(to):
    "Say Hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Christophe")

'Hello Christophe!'

In [ ]:
#hide
def special_parse(line, second_separator):
    """This is used for raw parsing of a text file, created manually 
    from a pdf file with a list of chinese words.
    It returns a dictionnary with hanzi, pinyin, and fr.
    """
    s1 = line.find(" ")
    rest = line[s1+1:]
    s2 = rest.find(second_separator)
    h = line[:s1]
    p = rest[0:s2]
    f = rest[s2+1:]
    return {"hanzi":h,"pinyin":p,"fr":f.strip()} 

In [ ]:
#hide
def initial_load():
    """Initial load of the raw text file"""
    print("start")
    total = 0
    terms = []
    with open("/home/chris/brain/1-projects/python-hsk2/data/raw.txt",'r')  as reader:
        for line in reader.readlines():
            # skip the lines 'Juin 2010'
            if(line.find("Juin")==-1):
                # find the first space
                s = line.find(' ')
                notraditional = line[s+1:]
                #print(line,end='')
                #print(notraditional, end='')
                if(line.find("#")!=-1):
                    r = special_parse(notraditional, "#")
                    print(r)
                    terms.append(r) 
                else:
                    r = special_parse(notraditional, " ")
                    print(r)
                    terms.append(r)
                total = total + 1
        print("ok")
    #print(terms)
    print("end")
    print("out of",total)
    return terms

In [ ]:
#hide
def create_study_plan(terms):
    """Create the initial study plan"""
    STUDYMAX = 30
    revised = []
    count = 0
    mydate = datetime.today()
    for term in terms:
        count += 1 
        if(count%STUDYMAX==0):
            mydate += timedelta(days=1)
        t = mydate.strftime('%Y-%m-%d')
        #print("add",t)
        term["study"] = t
        revised.append(term)
    return revised

In [ ]:
#export
def export_json(terms):
    with open("/home/chris/brain/1-projects/python-hsk2/data/study.json",'w')  as writer:
        json.dump(terms, writer)

In [ ]:
#export
def import_json():
    """Import the JSON back"""
    with open("/home/chris/brain/1-projects/python-hsk2/data/study.json",'r')  as reader:
        data = json.load(reader)
    return data

In [ ]:
#export 
def with_next_study_date(term,input):
    """Trivial learning algorithm: if there is an error: 
    we keep at the same day (no progress, will be asked the next time),
    if this is OK, then today+3 days"""
    mydate = datetime.today()
    if(input!="w"):
        mydate += timedelta(days=3)
        t = mydate.strftime('%Y-%m-%d')
        term["study"]=t
    return term

In [ ]:
#export
def play(terms):
    """Main Play Loop"""
    print("Let's play !!!!!!!!! (q to quit)")
    mydate = datetime.today().strftime('%Y-%m-%d')
    study_revised = []
    quit = False
    todo = 0
    for term in terms:
        date_time_obj = datetime.strptime(term["study"], '%Y-%m-%d')
        if(quit==False and date_time_obj <= datetime.today()):
            todo += 1
    counter = 1
    for term in terms:
        date_time_obj = datetime.strptime(term["study"], '%Y-%m-%d')
        if(quit==False and date_time_obj <= datetime.today()):
            print(counter, "/", todo, ": ", term["hanzi"], term["pinyin"], "?")  
            counter += 1 
            c=input("...")  
            if(c=="q"):
                quit = True
            else:
                print(term["fr"])
                c=input("...")  
                term = with_next_study_date(term,c)
                #print(term)
        study_revised.append(term)
    print("Well Done !!!!")
    return study_revised

And now the main:

In [ ]:
print("Start... 1 to init, 2 to play")
select = input()
if(select=="1"):
    terms = initial_load()
    revised = create_study_plan(terms)
    #print(revised)
    export_json(revised)
if(select=="2"):
    data = import_json()
    #print(data)
    study = play(data)
    export_json(study)
print("End")

Start... 1 to init, 2 to play
2
Let's play !!!!!!!!! (q to quit)
Well Done !!!!
End


### Let's try some string manipulations

Note that the string below have a u prefix : u indicates unicode string (whatever the encoding)

In [ ]:
h = u"还可以"
p = u"Hái kěyǐ"
f = u"acceptable"
myjson = {"hanzi":h,"pinyin":p,"fr":f}
myjson

{'hanzi': '还可以', 'pinyin': 'Hái kěyǐ', 'fr': 'acceptable'}

In [ ]:
json.dumps(myjson)

'{"hanzi": "\\u8fd8\\u53ef\\u4ee5", "pinyin": "H\\u00e1i k\\u011by\\u01d0", "fr": "acceptable"}'

## Some Asserts 

In [ ]:
assert say_hello("Christophe")=="Hello Christophe!"